# Example: Optimal adversaries for convolutional MNIST model

This notebook gives an example where OMLT is used to find adversarial examples for a trained convolutional neural network. We follow the below steps:<br>
1.) A convolutional neural network (CNN) with ReLU activation functions is trained to classify images from the MNIST dataset <br>
2.) OMLT is used to generate a mixed-integer encoding of the trained CNN using the big-M formulation <br>
3.) The model is optimized to find the maximum classification error (defined by an "adversarial" label) over a small input region <br>



## Library Setup
This notebook assumes you have a working PyTorch environment to train the neural network for classification. The neural network is then formulated in Pyomo using OMLT which therefore requires working Pyomo and OMLT installations.

The required Python libraries used this notebook are as follows: <br>
- `numpy`: used for manipulate input data <br>
- `torch`: the machine learning language we use to train our neural network
- `torchvision`: a package containing the MNIST dataset
- `pyomo`: the algebraic modeling language for Python, it is used to define the optimization model passed to the solver
- `onnx`: used to express trained neural network models
- `omlt`: the package this notebook demonstates. OMLT can formulate machine learning models (such as neural networks) within Pyomo

**NOTE:** This notebook also assumes you have a working MIP solver executable (e.g., CBC, Gurobi) to solve optimization problems in Pyomo. The open-source solver CBC is called by default.

In [4]:
#Import requisite packages
#data manipulation
import numpy as np
import tempfile

#pytorch for training neural network
import torch, torch.onnx
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

#pyomo for optimization
import pyomo.environ as pyo

#omlt for interfacing our neural network with pyomo
from omlt import OmltBlock
from omlt.neuralnet import FullSpaceNNFormulation
from omlt.io.onnx import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds

## Import the Data and Train a Neural Network

We begin by loading the MNIST dataset as `DataLoader` objects with pre-set training and testing batch sizes:

In [5]:
#set training and test batch sizes
train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}

#build DataLoaders for training and test sets
dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transforms.ToTensor())
dataset2 = datasets.MNIST('../data', train=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

Next, we define the structure of the convolutional neural network model:

In [6]:
hidden_size = 10

class Net(nn.Module):
    #define layers of neural network
    def __init__(self):
        super().__init__()
        self.conv1  = nn.Conv2d(1, 2, (4,4), (2,2), 0)
        self.conv2  = nn.Conv2d(2, 2, (4,4), (2,2), 0)
        self.hidden1 = nn.Linear(5*5*2, hidden_size)
        self.output  = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    #define forward pass of neural network
    def forward(self, x):
        self.x1 = self.conv1(x)
        self.x2 = self.relu(self.x1)
        self.x3 = self.conv2(self.x2)
        self.x4 = self.relu(self.x3)
        self.x5 = self.hidden1(self.x4.view((-1,5*5*2)))
        self.x6 = self.relu(self.x5)
        self.x7 = self.output(self.x6)
        x = self.softmax(self.x7)      
        return x

We next define simple functions for training and testing the neural network:

In [7]:
#training function computes loss and its gradient on batch, and prints status after every 200 batches
def train(model, train_loader, optimizer, epoch):
    model.train(); criterion = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

#testing function computes loss and prints overall model accuracy on test set
def test(model, test_loader):
    model.eval(); criterion = nn.NLLLoss(reduction='sum')
    test_loss = 0; correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))            

Finally, we train the neural network on the dataset.
Training here is performed using the `Adadelta` optimizer for five epochs.

In [8]:
#define model and optimizer
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

#train CNN model for five epochs
for epoch in range(5):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.292737
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.518931
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.740210
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.403113
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.711090

Test set: Average loss: 0.3681, Accuracy: 8816/10000 (88%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.387400
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.284508
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.549140
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.238617
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.328351

Test set: Average loss: 0.2623, Accuracy: 9197/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.103231
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.271127
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.539560
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.284458
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.139183

Test set: Average loss: 0.2452, Accuracy: 9254/10000 (93%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.248313
Train Epoch: 3 [12800/60000 (21%)]	Lo

## save the model 

In [9]:
# save the pytorch model 
import torch
torch.save(model.state_dict(), 'cnn_model.pth')